In [1]:
import pandas as pd
from pandas import DataFrame,Series
import numpy as np
import os
import datetime

#Plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

# sklearn stuff
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, mean_squared_error, precision_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, Imputer 
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

import feature_pipelines as pipes

### Submission Functions 

In [46]:
def generate_regression_preds(reg, model_name='pred_logerror'):
    reg_preds = None
    for i in range(int(properties.shape[0] / 100000)):   
        # get current test features
        current_test_feats = feature_pipeline.transform(properties.iloc[i*100000:(i+1)*100000])

        # predict on current test obs
        current_preds = Series(reg.predict(current_test_feats), name=model_name,
                              index = np.arange(i*100000,(i+1)*100000))

        if reg_preds is not None:
            reg_preds = pd.concat([reg_preds, current_preds])
        else:
            reg_preds = current_preds

    #  fencepost problem
    current_test_feats = feature_pipeline.transform(properties.iloc[2900000:])
    current_preds = Series(reg.predict(current_test_feats), name=model_name,
                          index = np.arange(2900000,2985217))
    reg_preds = pd.concat([reg_preds, current_preds])
    return reg_preds

In [3]:
def generate_submissions(oct_model,nov_model,dec_model,name='new_submission',logy=True):
    """
    This function creates the submission file for the public leaderboard predictions.
    Three already fitted models, one for each of the predicting time points, is required.
    """
    submission_df = DataFrame()
    for i in range(int(properties.shape[0] / 100000)):
        all_feats = full_pipeline.transform(properties.iloc[i*100000:(i+1)*100000])
        foo = properties.iloc[i*100000:(i+1)*100000][['parcelid']].reset_index(drop=True)
        if logy:
            foo = pd.concat([foo, DataFrame({'201610': oct_model.predict(all_feats),
                                                            '201611': nov_model.predict(all_feats),
                                                            '201612': dec_model.predict(all_feats)})], axis=1)
        else:
            foo = pd.concat([foo, DataFrame({'201610': np.log(oct_model.predict(all_feats)),
                                                            '201611': np.log(nov_model.predict(all_feats)),
                                                            '201612': np.log(dec_model.predict(all_feats))})], axis=1)
        submission_df = pd.concat([submission_df, foo], ignore_index=True)

    #  fencepost problem
    all_feats = full_pipeline.transform(properties.iloc[2900000:])
    foo = properties.iloc[2900000:][['parcelid']].reset_index(drop=True)
    foo = pd.concat([foo, DataFrame({'201610': oct_model.predict(all_feats),
                                                    '201611': nov_model.predict(all_feats),
                                                    '201612': dec_model.predict(all_feats)})], axis=1)
    submission_df = pd.concat([submission_df, foo], ignore_index=True)
    
    submission_df['201710'] = 0
    submission_df['201711'] = 0
    submission_df['201712'] = 0
    
    submission_df.rename(columns={'parcelid':'ParcelId'}, inplace=True)    
#     submission_df[['201610','201611','201612','201710','201711','201712']]= submission_df[['201610','201611','201612',
#                                                                                            '201710','201711','201712']].round(4)
    # unit test
    submission_df.drop_duplicates(inplace=True)
    assert submission_df.shape[0] == properties.shape[0]
    # write to .csv
    submission_df[['ParcelId','201610','201611','201612',
                  '201710','201711','201712']].to_csv(name + ".gz", index=False, float_format='%.4g', compression='gzip')
    return submission_df

In [4]:
def mean_absolute_errors(submission_df, comparison_df):
    """
    This function takes a submission entry for public leaderboard, and returns
    the training error for each month.
    """
    # training error
    trainresults = pd.merge(submission_df[['ParcelId','201610','201611','201612']], comparison_df[['parcelid','logerror','month']],
                           left_on='ParcelId', right_on='parcelid')
    oct_error = abs(trainresults[trainresults['month'] == 10]['201610'] 
                    - trainresults[trainresults['month'] == 10]['logerror']).mean()
    nov_error = abs(trainresults[trainresults['month'] == 11]['201611'] 
                    - trainresults[trainresults['month'] == 11]['logerror']).mean()
    dec_error = abs(trainresults[trainresults['month'] == 12]['201612'] 
                    - trainresults[trainresults['month'] == 12]['logerror']).mean()
    overall_mae = (oct_error*(trainresults['month'] == 10).sum() + nov_error*(trainresults['month'] == 11).sum() 
                        + dec_error*(trainresults['month'] == 12).sum()) / (trainresults['month'].isin([10,11,12])).sum()
    return (oct_error, nov_error, dec_error, overall_mae)

### Reading in data 

In [5]:
maindir = "/home/anerdi/Desktop/Zillow"

logerror = pd.read_csv(maindir + "/data/train_2016_v2.csv/train_2016_v2.csv")
logerror['weeknumber'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').isocalendar()[1])
logerror['month'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').month)
properties = pd.read_csv(maindir + "/data/properties_2016.csv/properties_2016.csv")

/home/anerdi/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
#proportion of living area
properties['N-LivingAreaProp'] = properties['calculatedfinishedsquarefeet']/properties['lotsizesquarefeet']

#Ratio of the built structure value to land area
properties['N-ValueProp'] = properties['structuretaxvaluedollarcnt']/properties['landtaxvaluedollarcnt']

#Ratio of tax of property over parcel
properties['N-ValueRatio'] = properties['taxvaluedollarcnt']/properties['taxamount']

# Pool
properties['Pool'] = (properties['pooltypeid2'].fillna(0) + properties['pooltypeid7'].fillna(0)).astype(int)

In [7]:
# join on parcel id
data = pd.merge(properties,logerror[['parcelid','logerror','month']], on='parcelid')

### Data  Preprocessing Pipeline

In [25]:
# Setup variables considered in the model

# numerical variables
# num_atts = ['calculatedfinishedsquarefeet','bathroomcnt','structuretaxvaluedollarcnt',
#              'bedroomcnt','calculatedbathnbr','roomcnt','longitude','threequarterbathnbr', 'yardbuildingsqft17',
#              'numberofstories','N-ValueRatio','N-ValueProp','N-LivingAreaProp']

num_atts = ['calculatedfinishedsquarefeet','bathroomcnt','structuretaxvaluedollarcnt',
             'bedroomcnt','calculatedbathnbr','N-ValueRatio','N-ValueProp','N-LivingAreaProp']

# categorical varaibles
cat_atts = ['airconditioningtypeid','heatingorsystemtypeid','Pool','propertylandusetypeid','taxdelinquencyflag',
           'regionidcounty']

# Dictionary of categorical variables and their default levels
cat_dict = {key:value for key,value in {'airconditioningtypeid':[-1] + list(range(1,14)),
           'architecturalstyletypeid':[-1] + list(range(1,28)),
           'buildingclasstypeid':[-1] + list(range(1,6)),
            'heatingorsystemtypeid':[-1] + list(range(1,26)),
            'pooltypeid10': list(range(-1,2)),
            'pooltypeid2': list(range(-1,2)),
            'pooltypeid7': list(range(-1,2)),
            'Pool': [0,1],
            'propertylandusetypeid': [-1, 31,46,47,246,247,248,260,261,262,263,264,265,266,267,268,269,270,271,
                                     273,274,275,276,279,290,291],
            'regionidcounty': [2061,3101,1286],
            'storytypeid':[-1] + list(range(1,36)),
            'typeconstructiontypeid':[-1] + list(range(1,19)),
            'yearbuilt': [-1] + list(range(1885,2018)),
            'fireplaceflag': [-1] + ['True','False'],
            'taxdelinquencyflag': [-1] + ['Y','N']
           }.items() if key in cat_atts}

# pairs to interact (x1,x2) where x1 is categorical and x2 is continuous
interact_pairs = [('regionidcounty','bathroomcnt'),('regionidcounty','bedroomcnt')
                 ,('regionidcounty','structuretaxvaluedollarcnt')]

In [26]:
# Categorical pipeline
cat_pipeline = Pipeline([
        ('select_and_dummify', pipes.DF_Selector_GetDummies(cat_dict)),
    ])

# Numerical pipeline
num_pipeline = Pipeline([
        ('selector', pipes.DataFrameSelector(num_atts)),
        ('imputer', Imputer()),
    ])

# interaction pipeline
interact_pipeline = Pipeline([
        ('dummify_and_interact',pipes.Dummify_and_Interact(interact_pairs,cat_dict)),
    ])

# Full pipeline
feature_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
        ("interact_pipeline", interact_pipeline)
    ])

In [27]:
feature_pipeline.fit(properties) #fitting the pipeline to the entire properties dataframe

FeatureUnion(n_jobs=1,
       transformer_list=[('num_pipeline', Pipeline(memory=None,
     steps=[('selector', DataFrameSelector(desired_cols=['calculatedfinishedsquarefeet', 'bathroomcnt', 'structuretaxvaluedollarcnt', 'bedroomcnt', 'calculatedbathnbr', 'N-ValueRatio', 'N-ValueProp', 'N-LivingAreaProp'])), ('imputer', Imputer(...roomcnt'), ('regionidcounty', 'bedroomcnt'), ('regionidcounty', 'structuretaxvaluedollarcnt')]))]))],
       transformer_weights=None)

In [28]:
import gc

In [29]:
gc.collect()

6

## Splitting the Training Set

In [53]:
ix_overestimated = np.where(data['logerror'] >= 0)[0]
ix_underestimated = np.where(data['logerror'] < 0)[0]
data_indices = {"over": ix_overestimated, "under": ix_underestimated}

In [54]:
assert ix_overestimated.shape[0] + ix_underestimated.shape[0] == data.shape[0]

## Training Elastic Net

In [76]:
from sklearn.linear_model import ElasticNet, Lars, HuberRegressor
from sklearn.base import clone

import warnings
warnings.filterwarnings("ignore")

In [33]:
models = [
    ("ridge",ElasticNet(alpha=1.25, l1_ratio = 0, max_iter=1000)),
    ("enet", ElasticNet(alpha=1.25, l1_ratio = 0.5, max_iter=1000)),
    ("lasso", ElasticNet(alpha=1.25, l1_ratio = 1, max_iter=1000)),
    ("larm", Lars(n_nonzero_coefs = 1)),
    ("huber", HuberRegressor())
]

In [77]:
test_predictions = pd.read_csv(maindir + "/data/properties_2016.csv/properties_2016.csv", usecols=['parcelid'])

for pair in models:
    current_model_name,current_model = pair
    print("Current model: %s" % current_model_name)
    
    for key,val in data_indices.items():
        type_of_zestimate, ix = key, val

        # preprocess current training data
        current_traindata = data.iloc[ix,]

        # get a clone of the model and fit the current training data
        reg = clone(current_model)
        reg.fit(feature_pipeline.transform(current_traindata), current_traindata['logerror'])

        # obtain predictions on test set
        reg_preds = generate_regression_preds(reg, model_name="%s_%s" % (current_model_name,type_of_zestimate))
        test_predictions = pd.concat([test_predictions, reg_preds], axis=1)

Current model: ridge
Current model: enet
Current model: lasso
Current model: larm
Current model: huber


In [78]:
test_predictions.head()

,parcelid,ridge_over,ridge_under,enet_over,enet_under,lasso_over,lasso_under,larm_over,larm_under,huber_over,huber_under
0,10754147,0.084871,-0.082879,0.073578,-0.068505,0.073431,-0.068225,0.071395,-0.064499,0.063218,-0.062623
1,10759547,0.085475,-0.082642,0.073578,-0.068505,0.073431,-0.068225,0.071395,-0.064499,0.063218,-0.062623
2,10843547,0.930269,-0.876458,0.558336,-0.414612,0.457031,-0.301205,0.071395,-0.064499,2.897348,-2.651653
3,10859147,0.121058,-0.124558,0.098928,-0.091456,0.095834,-0.087705,0.071395,-0.064499,0.169290,-0.171291
4,10879947,0.083832,-0.084374,0.073467,-0.068765,0.073493,-0.068667,0.071395,-0.064499,0.059819,-0.060274


In [58]:
overestimate_probabilities = pd.read_csv("/home/anerdi/Desktop/Zillow/twostagemodel/overestimate_probs.csv")

In [59]:
overestimate_probabilities.head()

,parcelid,overestimate_prob
0,10754147,0.509328
1,10759547,0.507312
2,10843547,1.000000
3,10859147,0.689618
4,10879947,0.543763


#### Merging Results

In [79]:
test_predictions = pd.merge(test_predictions, overestimate_probabilities, on='parcelid')

In [82]:
for pair in models:
    current_model_name, current_model = pair
    # combine over and under to get prediction
    test_predictions[current_model_name] = (test_predictions['%s_over' % current_model_name]*test_predictions['overestimate_prob'] +
                    test_predictions['%s_under' % current_model_name]*(1 - test_predictions['overestimate_prob']))

In [83]:
test_predictions.head()

,parcelid,ridge_over,ridge_under,enet_over,enet_under,lasso_over,lasso_under,larm_over,larm_under,huber_over,huber_under,overestimate_prob,ridge,enet,lasso,larm,huber
0,10754147,0.084871,-0.082879,0.073578,-0.068505,0.073431,-0.068225,0.071395,-0.064499,0.063218,-0.062623,0.509328,0.002561,0.003862,0.003925,0.004716,0.001471
1,10759547,0.085475,-0.082642,0.073578,-0.068505,0.073431,-0.068225,0.071395,-0.064499,0.063218,-0.062623,0.507312,0.002646,0.003575,0.003639,0.004442,0.001217
2,10843547,0.930269,-0.876458,0.558336,-0.414612,0.457031,-0.301205,0.071395,-0.064499,2.897348,-2.651653,1.000000,0.930269,0.558336,0.457031,0.071395,2.897348
3,10859147,0.121058,-0.124558,0.098928,-0.091456,0.095834,-0.087705,0.071395,-0.064499,0.169290,-0.171291,0.689618,0.044823,0.039836,0.038867,0.029216,0.063580
4,10879947,0.083832,-0.084374,0.073467,-0.068765,0.073493,-0.068667,0.071395,-0.064499,0.059819,-0.060274,0.543763,0.007090,0.008575,0.008634,0.009395,0.005028


In [93]:
model_name = 'lasso'
new_submission = DataFrame({'ParcelId': test_predictions['parcelid'],
                           '201610':test_predictions[model_name],
                           '201611':test_predictions[model_name],
                           '201612':test_predictions[model_name],
                           '201710':test_predictions[model_name],
                           '201711':test_predictions[model_name],
                           '201712':test_predictions[model_name]})

In [94]:
# two-stage with lasso
mean_absolute_errors(new_submission.round(4), data)

(0.06321498894916613,
 0.06216084337349388,
 0.07480948821161598,
 0.065350081948021527)

In [91]:
# two-stage with enet
mean_absolute_errors(new_submission.round(4), data)

(0.06320811733976288,
 0.06216155531215774,
 0.07480471535365159,
 0.065345258721610872)

In [88]:
# two-stage with larm
mean_absolute_errors(new_submission.round(4), data)

(0.06324942736588297,
 0.06221018619934285,
 0.07484307073030456,
 0.065387532193865519)

In [85]:
# two-stage with huber
mean_absolute_errors(new_submission.round(4), data)

(0.0633122161944946,
 0.06246286966046,
 0.07484818861414623,
 0.065479173495668438)

In [74]:
# two-stage with ridge
mean_absolute_errors(new_submission.round(4), data)

(0.06321432589913599,
 0.06228148959474268,
 0.07486952271420345,
 0.06538770779676889)

In [95]:
new_submission.round(4).to_csv("/home/anerdi/Desktop/Zillow/submissions/two_stage_lasso.csv.gz", index=False,
                     compression='gzip')